In [ ]:
!pip install simpletransformers


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.5/250.5 KB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 34.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 51.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 KB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 32.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 43.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 KB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 8.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import torch
import torch.nn as nn
import numpy as np 
import shutil
import sys
import requests

In [ ]:
df=pd.read_csv("/content/drive/MyDrive/Nane&Limon/Dataset/teknofest_train_final.csv",delimiter='|')

In [ ]:
url = "https://cryptic-oasis-68424.herokuapp.com/preprocess?tr_chars=false&acc_marks=true&punct=true&lower=true&offensive=false&norm_numbers=true&remove_numbers=false&remove_spaces=true&remove_stopwords=false&min_len=4"
texts = df.text.values.tolist()
preprocess_response = requests.post(url, json={"texts": texts})
processed_text = preprocess_response.json()['result']
df.text = processed_text
print(df[df.text == ''].sum())
df = df[df['text'] != '']
print(df.head())
print(df[df.text == ''].sum())

id              e2d954b7-266d-43be-845e-015a8ecf1241697c1629-d...
text                                                             
is_offensive                                                   79
target          RACISTRACISTRACISTRACISTSEXISTOTHEROTHEROTHERR...
dtype: object
                                     id  \
0  81c11060-a240-4d54-841b-9e2916039e85   
1  be80ebbf-b322-4c3b-afa1-94932ea80731   
2  f99e2513-83ed-4076-ac72-b9e2cff3f049   
3  83ed2b2e-b815-4f36-9fc4-80a9050cf2d0   
4  d93e05f7-bfdd-4cdb-99d8-3048761b30ff   

                                                text  is_offensive     target  
0                                        çürük dişli             1     INSULT  
1  bu adamın islama ve müslümanlara verdiği zarar...             1     RACIST  
2                               erkekler zora gelmez             1     SEXIST  
3  utanmazın götüne kazık sokmuşlar bu tıkırtı ne...             1  PROFANITY  
4       otomasyon sistemlerine doğrudan bağlanabilir           

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12467 entries, 0 to 12616
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            12467 non-null  object
 1   text          12467 non-null  object
 2   is_offensive  12467 non-null  int64 
 3   target        12467 non-null  object
dtypes: int64(1), object(3)
memory usage: 487.0+ KB


In [ ]:
df_dmy=pd.get_dummies(df.target)

In [ ]:
df=pd.concat([df.text, df_dmy], axis=1)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12467 entries, 0 to 12616
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   text       12467 non-null  object
 1   INSULT     12467 non-null  uint8 
 2   OTHER      12467 non-null  uint8 
 3   PROFANITY  12467 non-null  uint8 
 4   RACIST     12467 non-null  uint8 
 5   SEXIST     12467 non-null  uint8 
dtypes: object(1), uint8(5)
memory usage: 255.7+ KB


In [ ]:
target_list=['INSULT','OTHER','PROFANITY','RACIST','SEXIST']#ofensif mi durumu sonradan belirlenecek .. 

In [ ]:
train_size=0.8
train_df=df.sample(frac=train_size,random_state=52025).reset_index(drop=True)
val_df=df.drop(train_df.index).reset_index(drop=True)
#train_size=0.25
#test_df=val_df.sample(frac=train_size,random_state=52025).reset_index(drop=True)
#val_df=val_df.drop(test_df.index).reset_index(drop=True)

In [ ]:
val_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2493 entries, 0 to 2492
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   text       2493 non-null   object
 1   INSULT     2493 non-null   uint8 
 2   OTHER      2493 non-null   uint8 
 3   PROFANITY  2493 non-null   uint8 
 4   RACIST     2493 non-null   uint8 
 5   SEXIST     2493 non-null   uint8 
dtypes: object(1), uint8(5)
memory usage: 31.8+ KB


In [ ]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9974 entries, 0 to 9973
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   text       9974 non-null   object
 1   INSULT     9974 non-null   uint8 
 2   OTHER      9974 non-null   uint8 
 3   PROFANITY  9974 non-null   uint8 
 4   RACIST     9974 non-null   uint8 
 5   SEXIST     9974 non-null   uint8 
dtypes: object(1), uint8(5)
memory usage: 126.7+ KB


In [ ]:
#test_df.info()

In [ ]:
train_df[target_list].values

array([[0, 1, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [0, 0, 1, 0, 0],
       ...,
       [0, 0, 0, 0, 1],
       [1, 0, 0, 0, 0],
       [0, 0, 1, 0, 0]], dtype=uint8)

In [ ]:
#bazı hyperparametreler..
MAX_LEN=40
TRAIN_BATCH_SIZE=8
VALID_BATCH_SIZE=8
EPOCHS=7
LR=5e-5
LM = "dbmdz/bert-base-turkish-128k-uncased"

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from transformers import BertTokenizer, BertModel

In [ ]:
tokenizer = BertTokenizer.from_pretrained(LM, do_lower_case=True, use_cuda=torch.cuda.is_available())
tokenizer

BertTokenizer(name_or_path='dbmdz/bert-base-turkish-128k-uncased', vocab_size=128000, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

In [ ]:
exm=" elif ben sana ne demiştim kızım, çocuğu uyuturken uyuya kalma demedim mi # ? "
encodings=tokenizer.encode_plus(exm,
                                add_special_tokens=True,
                                max_length=MAX_LEN,
                                padding='max_length',
                                truncation=True,
                                return_attention_mask=True,
                                return_tensors='pt')

In [ ]:
encodings

{'input_ids': tensor([[    2, 12626,  2105,  3247,  2073, 71015,  3180,  9946,    16, 89973,
         27716,  5551, 21660,  6034, 30094,  2477,     7,    35,     3,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])}

In [ ]:
train_df['text'][0]

'ben canım sevgilimi ararım'

In [ ]:
target_list

['INSULT', 'OTHER', 'PROFANITY', 'RACIST', 'SEXIST']

In [ ]:
class CustomDataset(torch.utils.data.Dataset):

    def __init__(self, df, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.df = df
        self.title = df['text']
        self.targets = self.df[target_list].values
        self.max_len = max_len

    def __len__(self):
        return len(self.title)

    def __getitem__(self, index):
        title = str(self.title[index])
        title = " ".join(title.split())

        inputs = self.tokenizer.encode_plus(
            title,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_token_type_ids=True,
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )

        return {
            'input_ids': inputs['input_ids'].flatten(),
            'attention_mask': inputs['attention_mask'].flatten(),
            'token_type_ids': inputs["token_type_ids"].flatten(),
            'targets': torch.FloatTensor(self.targets[index])
        }

In [ ]:
train_dataset = CustomDataset(train_df, tokenizer, MAX_LEN)
valid_dataset = CustomDataset(val_df, tokenizer, MAX_LEN)

In [ ]:
train_data_loader = torch.utils.data.DataLoader(train_dataset, 
    batch_size=TRAIN_BATCH_SIZE,
    shuffle=True,
    num_workers=0
)

val_data_loader = torch.utils.data.DataLoader(valid_dataset, 
    batch_size=VALID_BATCH_SIZE,
    shuffle=False,
    num_workers=0
)

In [ ]:
print(len(val_data_loader))

312


In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

device(type='cuda')

In [ ]:
def load_ckp(checkpoint_fpath, model, optimizer):
    """
    checkpoint_path: path to save checkpoint
    model: model that we want to load checkpoint parameters into       
    optimizer: optimizer we defined in previous training
    """
    # load check point
    checkpoint = torch.load(checkpoint_fpath)
    # initialize state_dict from checkpoint to model
    model.load_state_dict(checkpoint['state_dict'])
    # initialize optimizer from checkpoint to optimizer
    optimizer.load_state_dict(checkpoint['optimizer'])
    # initialize valid_loss_min from checkpoint to valid_loss_min
    valid_loss_min = checkpoint['valid_loss_min']
    # return model, optimizer, epoch value, min validation loss 
    return model, optimizer, checkpoint['epoch'], valid_loss_min.item()

def save_ckp(state, is_best, checkpoint_path, best_model_path):
    """
    state: checkpoint we want to save
    is_best: is this the best checkpoint; min validation loss
    checkpoint_path: path to save checkpoint
    best_model_path: path to save best model
    """
    f_path = checkpoint_path
    # save checkpoint data to the path given, checkpoint_path
    torch.save(state, f_path)
    # if it is a best model, min validation loss
    if is_best:
        best_fpath = best_model_path
        # copy that checkpoint file to best path given, best_model_path
        shutil.copyfile(f_path, best_fpath)

In [ ]:
class BERTClass(torch.nn.Module):
    def __init__(self):
        super(BERTClass, self).__init__()
        self.bert_model = BertModel.from_pretrained(LM, return_dict=True)
        self.dropout = torch.nn.Dropout(0.3) 
        self.linear = torch.nn.Linear(768, 5)
    
    def forward(self, input_ids, attn_mask, token_type_ids):
        output = self.bert_model(
            input_ids, 
            attention_mask=attn_mask, 
            token_type_ids=token_type_ids
        )
        output_dropout = self.dropout(output.pooler_output)
        output = self.linear(output_dropout)
        return output

model = BERTClass()
model.to(device)

Some weights of the model checkpoint at dbmdz/bert-base-turkish-128k-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BERTClass(
  (bert_model): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(128000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_

In [ ]:
#base modeli kaydediyoruz .. 
PATH='/content/drive/MyDrive/Nane&Limon/multilabel_prediction/mdl'
torch.save(model, PATH)
torch.save(trained_model, '/content/drive/MyDrive/Colab Notebooks/multilabel/trained_model')

In [ ]:
def loss_fn(outputs, targets):
    return torch.nn.BCEWithLogitsLoss()(outputs, targets)

optimizer = torch.optim.AdamW(params =  model.parameters(), lr=LR,eps = 1e-7)

In [ ]:
val_targets=[]
val_outputs=[]
     

In [ ]:
def train_model(n_epochs, training_loader, validation_loader, model, 
                optimizer, checkpoint_path, best_model_path):
  print(len(training_loader))
  print(len(validation_loader))
  # initialize tracker for minimum validation loss
  valid_loss_min = np.Inf
   
 
  for epoch in range(1, n_epochs+1):
    train_loss = 0
    valid_loss = 0

    model.train()
    print('############# Epoch {}: Training Start   #############'.format(epoch))
    for batch_idx, data in enumerate(training_loader):
        #print('yyy epoch', batch_idx)
        ids = data['input_ids'].to(device, dtype = torch.long)
        mask = data['attention_mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.float)

        outputs = model(ids, mask, token_type_ids)

        optimizer.zero_grad()
        loss = loss_fn(outputs, targets)
        #if batch_idx%5000==0:
         #   print(f'Epoch: {epoch}, Training Loss:  {loss.item()}')
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        #print('before loss data in training', loss.item(), train_loss)
        train_loss = train_loss + ((1 / (batch_idx + 1)) * (loss.item() - train_loss))
        #print('after loss data in training', loss.item(), train_loss)
    
    print('############# Epoch {}: Training End     #############'.format(epoch))
    
    print('############# Epoch {}: Validation Start   #############'.format(epoch))
    ######################    
    # validate the model #
    ######################
 
    model.eval()
   
    with torch.no_grad():
      for batch_idx, data in enumerate(validation_loader, 0):
            ids = data['input_ids'].to(device, dtype = torch.long)
            mask = data['attention_mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.float)
            outputs = model(ids, mask, token_type_ids)

            loss = loss_fn(outputs, targets)
            valid_loss = valid_loss + ((1 / (batch_idx + 1)) * (loss.item() - valid_loss))
            val_targets.extend(targets.cpu().detach().numpy().tolist())
            val_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())
            '''detach() detaches from the backward graph to avoid copying gradients.
               cpu() moves the data to CPU.
               numpy() converts the torch.Tensor to a np.ndarray.'''

      print('############# Epoch {}: Validation End     #############'.format(epoch))
      # calculate average losses
      #print('before cal avg train loss', train_loss)
      train_loss = train_loss/len(training_loader)
      #print(train_loss)
      valid_loss = valid_loss/len(validation_loader)
      #print(valid_loss)
      # print training/validation statistics 
      print('Epoch: {} \tAvgerage Training Loss: {:.6f} \tAverage Validation Loss: {:.6f}'.format(
            epoch, 
            train_loss,
            valid_loss
            ))
      
      # create checkpoint variable and add important data
      checkpoint = {
            'epoch': epoch + 1,
            'valid_loss_min': valid_loss,
            'state_dict': model.state_dict(),
            'optimizer': optimizer.state_dict()
      }
        
        # save checkpoint
      save_ckp(checkpoint, False, checkpoint_path, best_model_path)
        
      ## TODO: save the model if validation loss has decreased
      if valid_loss <= valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(valid_loss_min,valid_loss))
        # save checkpoint as best model
        save_ckp(checkpoint, True, checkpoint_path, best_model_path)
        valid_loss_min = valid_loss

    print('############# Epoch {}  Done   #############\n'.format(epoch))

  return model

In [ ]:
ckpt_path = "/content/drive/MyDrive/Nane&Limon/multilabel_prediction/curr_ckpt"
best_model_path = "/content/drive/MyDrive/Nane&Limon/multilabel_prediction/best_model.pt"

In [ ]:
trained_model = train_model(EPOCHS, train_data_loader, val_data_loader, model, optimizer, ckpt_path, best_model_path)

1247
312
############# Epoch 1: Training Start   #############
############# Epoch 1: Training End     #############
############# Epoch 1: Validation Start   #############
############# Epoch 1: Validation End     #############
Epoch: 1 	Avgerage Training Loss: 0.000132 	Average Validation Loss: 0.000268
Validation loss decreased (inf --> 0.000268).  Saving model ...
############# Epoch 1  Done   #############

############# Epoch 2: Training Start   #############
############# Epoch 2: Training End     #############
############# Epoch 2: Validation Start   #############
############# Epoch 2: Validation End     #############
Epoch: 2 	Avgerage Training Loss: 0.000060 	Average Validation Loss: 0.000171
Validation loss decreased (0.000268 --> 0.000171).  Saving model ...
############# Epoch 2  Done   #############

############# Epoch 3: Training Start   #############
############# Epoch 3: Training End     #############
############# Epoch 3: Validation Start   #############
########

In [ ]:
type(trained_model)

__main__.BERTClass

In [ ]:
torch.save(trained_model, '/content/drive/MyDrive/Nane&Limon/multilabel_prediction/trained_model')

In [ ]:
saved_model = torch.load('/content/drive/MyDrive/Nane&Limon/multilabel_prediction/trained_model')

In [ ]:
saved_model.eval()

BERTClass(
  (bert_model): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(128000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_

In [ ]:
model = saved_model

for param in model.parameters():
  print(param)

Görüntülenen çıkış son 5000 satıra kısaltıldı.
        -5.0245e-02,  2.5189e-02, -9.3486e-03, -1.3739e-02,  1.6566e-02,
         6.8452e-03,  1.7038e-02,  3.6514e-02,  6.5135e-02, -1.1136e-02,
        -5.3641e-03,  4.7097e-02, -2.3739e-02, -2.4890e-02, -1.0008e-01,
         2.5638e-03, -5.9546e-03, -2.1611e-02,  3.3651e-03,  1.3148e-01,
         2.3326e-02, -3.0938e-02,  5.2052e-02,  1.4837e-02,  7.6266e-02,
         1.5632e-02, -1.5370e-02,  3.2544e-02, -5.1220e-02,  1.0691e-02,
        -2.3036e-02, -4.5943e-02,  2.9538e-02, -2.3830e-02,  9.7675e-03,
        -4.5583e-02,  2.1176e-02,  1.8080e-02,  4.1240e-02, -5.5868e-04,
        -2.9759e-02, -9.2633e-03,  2.5770e-02, -1.7120e-02, -1.3064e-02,
         1.0584e-02,  6.7324e-03, -3.0887e-02, -7.9151e-03, -8.9859e-04,
        -7.4846e-03,  1.4407e-02,  1.0551e-02,  4.8933e-03,  6.5125e-03,
         8.8646e-03,  4.9484e-03, -4.5938e-03,  2.3131e-02,  5.6099e-04,
        -7.5008e-04, -3.2509e-02,  2.6873e-02,  4.0252e-02, -2.8423e-02,
    

In [ ]:
df_shap_ex = pd.read_csv("/content/drive/MyDrive/Nane&Limon/Dataset/shap.csv")

In [ ]:
df_shap_ex.drop("Unnamed: 0", axis=1, inplace=True)

In [ ]:
df_shap_ex.head()

,text,target
0,*Sakallı lavuk trans kız olduğunu iddia eden t...,RACIST
1,48 kilo kizmi olur amk kadın dediğin ele avuca...,PROFANITY
2,Ay bokum 1.60 altı kadınlar kendine kadınım de...,PROFANITY
3,Bakire kız arıyorsan sen de bakir erkek olacak...,PROFANITY
4,Berlin düşüp faşizm kaybettikten sonra Kızıl O...,RACIST


In [ ]:
predicted = []
for text in df_shap_ex['text']:
    encodings = tokenizer.encode_plus(
        text,
        None,
        add_special_tokens=True,
        max_length=MAX_LEN,
        padding='max_length',
        return_token_type_ids=True,
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt'
    )

    with torch.no_grad():
        input_ids = encodings['input_ids'].to(device, dtype=torch.long)
        attention_mask = encodings['attention_mask'].to(device, dtype=torch.long)
        token_type_ids = encodings['token_type_ids'].to(device, dtype=torch.long)
        output = model(input_ids, attention_mask, token_type_ids)
        final_output = torch.sigmoid(output).cpu().detach().numpy().tolist()
        predicted.append(train_df.columns[1:].to_list()[int(np.argmax(final_output, axis=1))])

df_shap_ex['predicted'] = predicted

In [ ]:
df_shap_ex

,text,target,predicted
0,*Sakallı lavuk trans kız olduğunu iddia eden t...,RACIST,SEXIST
1,48 kilo kizmi olur amk kadın dediğin ele avuca...,PROFANITY,SEXIST
2,Ay bokum 1.60 altı kadınlar kendine kadınım de...,PROFANITY,SEXIST
3,Bakire kız arıyorsan sen de bakir erkek olacak...,PROFANITY,SEXIST
4,Berlin düşüp faşizm kaybettikten sonra Kızıl O...,RACIST,OTHER
5,biz bu hayatta hic esmer 1.80+ yakisikli cocuk...,OTHER,INSULT
6,Çocuğun şu ortama girmek için yalandan dinledi...,PROFANITY,SEXIST
7,Erkek adam onu ben hallederim deyip söküp daha...,PROFANITY,PROFANITY
8,Lezbiyen ya da aldatılmış sivilceli şişman bı ...,PROFANITY,INSULT
9,Geldi yine tibini fofenklediim top sakallı gız,PROFANITY,INSULT
